In [11]:
import pandas as pd
import numpy as np
import time

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor

from sklearn.metrics import root_mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import joblib

In [12]:
df = pd.read_parquet('../04_EDA/car_ads_model.parquet')

In [13]:
df.shape

(182826, 20)

In [14]:
# Function to apply polynomial regression and add predicted prices
def reg_outliers(group):
    # Independent variable (e.g., km)
    X = group[['km']]
    y = group['price']  # Dependent variable

    # Polynomial transformation (degree 3)
    poly = PolynomialFeatures(degree=3)
    X_poly = poly.fit_transform(X)
    
    # Fit the linear regression model
    model = LinearRegression()
    model.fit(X_poly, y)
    
    # Make predictions
    y_pred = model.predict(X_poly)
    
    # Assign the predicted price to a new column
    group['reg_price'] = y_pred.astype(int)
    
    return group

In [15]:
# Assuming 'df' is your original DataFrame
df = df.groupby(['brand', 'model']).apply(reg_outliers).reset_index(drop=True)

C:\Users\samue\AppData\Local\Temp\ipykernel_6908\909315548.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(['brand', 'model']).apply(reg_outliers).reset_index(drop=True)


In [16]:
df['reg_diff_abs'] = (df['price'] - df['reg_price']).abs().astype(int)

In [17]:
df = df[((df['reg_diff_abs'] / df['price']) < 1) | (df['reg_diff_abs'] < 7_000)].reset_index(drop=True)

In [18]:
# Step 2: Lists to store results
all_results = []
performance_metrics = []

In [19]:
# Function to encode 'cv' (horsepower) column
def encode_cv(group):
    # Drop null values and convert to int for categorical representation
    non_null_cv = group['cv'].dropna().astype(int)

    # Create categorical labels
    categories = non_null_cv.unique()
    cat_labels = [f'cv_{cat}' for cat in categories]

    # Create a new DataFrame with one-hot encoding for existing categories
    cv_encoded = pd.get_dummies(non_null_cv, prefix='', prefix_sep='', dtype=int)

    # Rename columns to desired format
    cv_encoded.columns = [f'cv_{col}' for col in cv_encoded.columns]

    # Return the encoded DataFrame
    return cv_encoded


# Function to encode 'fuel' and other categorical variables
def encode_categorical_features(group):
    # One-hot encoding for 'fuel'
    fuel_encoded = pd.get_dummies(group['fuel'], prefix='fuel')
    cv_encoded = encode_cv(group)
    # # Handle CV encoding if 'cv' is present
    # if group['cv'].notnull().all():
    #     cv_encoded = encode_cv(group)
    # else:
    #     cv_encoded = pd.DataFrame(index=group.index)  # Empty if no 'cv'
    
    # Combine encoded features
    return pd.concat([fuel_encoded, cv_encoded], axis=1)

# Function to select a model based on the number of samples
def select_model(n_samples):
    if n_samples < 30:
        return SVR(kernel='rbf', C=0.5, epsilon=0.01)  # L2 Regularization
    elif n_samples < 2_000:
        return XGBRegressor(nthread=-1,
                            n_estimators=500,
                            learning_rate=0.01,
                            min_child_weight=5,
                            subsample=0.8,
                            colsample_bytree=0.8,
                            max_depth=5,
                            monotone_constraints={'km': -1, 'age_years':-1})  # Tree-based regularization
    else:
        return RandomForestRegressor(n_estimators=1000, min_samples_leaf=10, max_depth=7, n_jobs=-1)  # Tree-based with max depth


# Function to choose and apply model
def choose_model(group):
    # Start the timer
    start_time = time.time()
    X = group[['km', 'age_years', 'is_automatic']]  # Base features
    
    # Encode categorical features (fuel and cv)
    cat_encoded = encode_categorical_features(group)
    X = pd.concat([X, cat_encoded], axis=1)

    # Ensure that there are no NaN values in X
    if X.isnull().values.any():
        # You can choose to fill NaNs or drop them
        X.fillna(0, inplace=True)  # Example: fill NaNs with 0
        
    # Get target variable
    y = group['price']  # Target (Price)
    n_samples = len(group)
    
    # If very few samples, predict the mean price
    if n_samples < 5:
        group['predicted_price'] = [group['price'].median()] * n_samples
        group['price_diff'] = group['price'] - group['predicted_price']
        rmse = root_mean_squared_error(y, group['predicted_price'])
        mape = mean_absolute_percentage_error(y, group['predicted_price'])
        mae = mean_absolute_error(y, group['predicted_price'])
        performance_metrics.append({
            'brand': group['brand'].iloc[0],
            'model': group['model'].iloc[0],
            'rmse': rmse,
            'mape': mape,
            'mae':mae
        })
        all_results.append(group)
        return

    # Train-test split logic
    test_size = 0.3 if n_samples >= 40 else 0.4
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

    # Select the model based on dataset size
    model = select_model(n_samples)
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Predicting on test set
    y_pred_test = model.predict(X_test)
    
    # Train on the entire dataset to predict for all data
    model.fit(X, y)
    y_pred_all = model.predict(X)

    # Save the trained model, feature names, and other metadata to disk
    brand = group['brand'].iloc[0]
    model_name = group['model'].iloc[0]
    model_file_name = f'saved_models/{brand}_{model_name}.pkl'
    
    # Create a dictionary to store model, feature names, and any other metadata
    model_info = {
        'model': model,
        'feature_names': X.columns.tolist(),  # Save the column names
        'brand': brand,
        'model_name': model_name,
        'training_samples': n_samples  # Optional metadata
    }
    
    # Save the model and additional info
    joblib.dump(model_info, model_file_name)
    
    # Calculate performance metrics
    rmse = root_mean_squared_error(y_test, y_pred_test)
    mape = mean_absolute_percentage_error(y_test, y_pred_test) * 100
    mae = mean_absolute_error(y_test, y_pred_test)
    performance_metrics.append({
        'brand': brand,
        'model': model_name,
        'rmse': round(rmse, 2),
        'mape': round(mape, 2),
        'mae': int(mae)
    })

    # Add predictions for all data points
    group['predicted_price'] = y_pred_all
    group['price_diff'] = group['price'] - group['predicted_price']
    
    all_results.append(group)

    elapsed_time = time.time() - start_time
    print(f"Car Model: {brand} {model_name}, Training Time: {elapsed_time:.2f} seconds")

In [20]:
df.groupby(['brand','model']).apply(choose_model, include_groups=True)

Car Model: abarth 500, Training Time: 13.07 seconds
Car Model: abarth 500c, Training Time: 9.84 seconds
Car Model: aiways u5, Training Time: 0.01 seconds
Car Model: alfa romeo 147, Training Time: 13.07 seconds
Car Model: alfa romeo 156, Training Time: 5.02 seconds
Car Model: alfa romeo 159, Training Time: 7.61 seconds
Car Model: alfa romeo brera, Training Time: 0.02 seconds
Car Model: alfa romeo giulia, Training Time: 14.23 seconds
Car Model: alfa romeo giulietta, Training Time: 13.70 seconds
Car Model: alfa romeo gt, Training Time: 5.12 seconds
Car Model: alfa romeo gtv, Training Time: 0.01 seconds
Car Model: alfa romeo mito, Training Time: 12.54 seconds
Car Model: alfa romeo spider, Training Time: 0.01 seconds
Car Model: alfa romeo stelvio, Training Time: 14.76 seconds
Car Model: alfa romeo tonale, Training Time: 9.74 seconds
Car Model: alpine a110, Training Time: 0.01 seconds
Car Model: asia rocsta, Training Time: 0.01 seconds
Car Model: aston martin db9, Training Time: 0.01 seconds

C:\Users\samue\AppData\Local\Temp\ipykernel_6908\237710914.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby(['brand','model']).apply(choose_model, include_groups=True)


""


In [26]:
results_df = pd.DataFrame(performance_metrics)  # Model performance metrics
final_results_df = pd.concat(all_results, ignore_index=True)  # Test data with predictions and price diff

In [27]:
final_cols_order = ['price','predicted_price', 'price_diff', 'brand', 'model', 'is_automatic','km', 'fuel',
                    'year', 'age_years', 'cv', 'kw', 'body_type', 'cubic_capacity', 'is_seller_pro','is_certified', 'offer_type',
                    'location', 'ad_days_creation', 'ad_days_published', 'title']

In [28]:
final_results_df = final_results_df[final_cols_order]

numeric_cols_list = final_results_df.select_dtypes(include='number').columns.values
final_results_df[numeric_cols_list] =  final_results_df[numeric_cols_list].round(0).astype('Int32')

In [30]:
results_df['rmse'].describe(percentiles=np.arange(0,1.1,0.1)).astype(int)

count     1202
mean      3698
std       4950
min          0
0%           0
10%        250
20%       1037
30%       1382
40%       1725
50%       2223
60%       2786
70%       3626
80%       5271
90%       8528
100%     54329
max      54329
Name: rmse, dtype: int64

In [31]:
final_results_df['price_diff'].abs().describe(percentiles=np.arange(0,1.1,0.1)).astype(int)

count    177440
mean       1722
std        2652
min           0
0%            0
10%         161
20%         336
30%         528
40%         742
50%         991
60%        1297
70%        1712
80%        2373
90%        3757
100%      98889
max       98889
Name: price_diff, dtype: int64

In [32]:
overall_accuracy = (100 * (1- mean_absolute_percentage_error(final_results_df['price'], final_results_df['predicted_price']))).round(2)

In [33]:
f"Model overall accuracy: {overall_accuracy}%"

'Model overall accuracy: 84.94%'

In [34]:
overall_error = mean_absolute_error(final_results_df['price'], final_results_df['predicted_price']).astype(int)

In [35]:
f"Model overall price-vs-prediction difference: {overall_error}€"

'Model overall price-vs-prediction difference: 1722€'

In [36]:
root_mean_squared_error(final_results_df['price'], final_results_df['predicted_price']).astype(int)

np.int64(3162)

In [37]:
performance_df = pd.DataFrame(performance_metrics)

In [38]:
def get_price_iqr(group, high, low):
    perc_high, perc_low = np.percentile(group['price_diff'], [high , low])
    return round(perc_high - perc_low, -2).astype(int)

In [39]:
iqr_df_25_75 = final_results_df.groupby(['brand','model']).apply(get_price_iqr, high=75, low=25).reset_index(name='iqr_25_75')

C:\Users\samue\AppData\Local\Temp\ipykernel_6908\3502448637.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  iqr_df_25_75 = final_results_df.groupby(['brand','model']).apply(get_price_iqr, high=75, low=25).reset_index(name='iqr_25_75')


In [40]:
iqr_df_10_90 = final_results_df.groupby(['brand','model']).apply(get_price_iqr, high=90, low=10).reset_index(name='iqr_10_90')

C:\Users\samue\AppData\Local\Temp\ipykernel_6908\1153704430.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  iqr_df_10_90 = final_results_df.groupby(['brand','model']).apply(get_price_iqr, high=90, low=10).reset_index(name='iqr_10_90')


In [41]:
performance_df = performance_df.merge(iqr_df_25_75, left_on=['brand','model'], right_on=['brand','model'])

In [42]:
performance_df = performance_df.merge(iqr_df_10_90, left_on=['brand','model'], right_on=['brand','model'])

In [43]:
performance_df

,brand,model,rmse,mape,mae,iqr_25_75,iqr_10_90
0,abarth,124 spider,8507.997708,0.750436,6897.5,9400,17300
1,abarth,500,1594.850000,6.710000,1207.0,1500,2900
2,abarth,500c,2805.970000,11.870000,1947.0,1500,3700
3,abarth,695,3227.744414,0.114280,2500.0,3800,6000
4,abarth,punto evo,0.000000,0.000000,0.0,0,0
...,...,...,...,...,...,...,...
1197,volvo,xc60,3427.680000,8.220000,2425.0,3400,7100
1198,volvo,xc70,2206.770000,14.400000,1483.0,1200,2700
1199,volvo,xc90,4389.080000,12.180000,2913.0,3100,7600
1200,yudo,3,5172.240000,20.430000,3836.0,7100,7700


In [44]:
# Save performance metrics to CSV
performance_df.to_csv('../app/app_files/performance_metrics.csv', index=False)

In [45]:
final_results_df.to_csv('../app/app_files/final_results_df.csv', index=False)

In [46]:
final_results_df = pd.read_csv('../app/app_files/final_results_df.csv')

In [47]:
performance_df = pd.read_csv('../app/app_files/performance_metrics.csv')